In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib.pyplot as plt

from models.migdal import Migdal
from detectors.sabre import SABRE
from veldists import VelDist
from constants import *

from scipy import stats

import multihist as mh
import pickle as pkl

In [2]:
def smearing_fn(energy):
    return np.sqrt(energy / 1000.) * 0.014 * 1000.

In [3]:
Det = SABRE()
Model = Migdal()
Dist = VelDist("avSHM", 0.3)

In [4]:
# masses_MeV = np.geomspace(200, 3000, 20)
masses_MeV = np.geomspace(50, 3000, 20)

masses_MeV = np.array([int(m) for m in masses_MeV])
ref_xsec = 1e-36

In [5]:
# binning = np.arange(1., 2.05, 0.05)
binning = np.arange(0.5, 2.05, 0.05)

In [6]:
for mass in masses_MeV:
    energy_edges_unsmeared = np.arange(0.01, 2., 0.01)
    rate_edges_unsmeared = np.array([Det.dRdE_True(e, Model.dRdER, mX=mass*MeV, sig=ref_xsec, VelDist=Dist,
                                                   NR=False,
                                                   SI=False, cp=1, cn=0, jx=1/2) for e in energy_edges_unsmeared])
    rate_centers_unsmeared = 0.5 * (rate_edges_unsmeared[1:] + rate_edges_unsmeared[:-1])
    hist_unsmeared = mh.Histdd.from_histogram(histogram=rate_centers_unsmeared, bin_edges=[energy_edges_unsmeared])
    
    sample = hist_unsmeared.get_random(int(1e7))[:, 0]
    sample_smear = stats.norm.rvs(loc=sample, scale=(smearing_fn(sample)))

    hist_smeared = np.histogram(sample_smear, bins=binning)[0]
    hist_smeared = mh.Histdd.from_histogram(histogram=hist_smeared, bin_edges=[binning])
    
    normalisation_unsmeared = (hist_unsmeared * hist_unsmeared.bin_volumes()).n
    fractional_loss = hist_smeared.n / len(sample_smear)
    
    hist_smeared.histogram = hist_smeared.histogram / (hist_smeared * hist_smeared.bin_volumes()).n * normalisation_unsmeared * fractional_loss
    
    pkl.dump(hist_smeared, open(f'SDWIMPMigdal_{mass}.pkl', 'wb'))